In [1]:
import os
import sys
sys.path.append('/home/opencv/src/models/research')
sys.path.append('/home/opencv/src/models/research/slim/')
if 'PYTHONPATH' not in os.environ:
    os.environ['PYTHONPATH'] = ""
os.environ['PYTHONPATH'] += ':/home/opencv/src/models/research:/home/opencv/src/models/research/slim/'

In [2]:
# For the fruit model included in the repo below we have 240 training images
# For faster training time, images should be resized to 300x300 and then annotated
# Images should contain the objects of interest at various scales, angles, lighting conditions, locations
# For acceptable results - mAP@0.5 of 0.9 the model was trained with batch size of 24
# and 5000 steps. this takes about 1h using 2 augmentations. 
# using 5 augmentations it takes about 2h 
num_steps = 5000  # A step means using a single batch of data. larger batch, less steps required
#Number of evaluation steps.
num_eval_steps = 50
#Batch size 24 is a setting that generally works well. can be changed higher or lower 
MODELS_CONFIG = {
        'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 24
    }
}
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colab's GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [3]:
!ls ~/src/object_detection_demo_flow/data/images/train

20200811_wooden_rgb_0000.jpg  20200811_wooden_rgb_0008.jpg
20200811_wooden_rgb_0000.xml  20200811_wooden_rgb_0008.xml
20200811_wooden_rgb_0001.jpg  20200811_wooden_rgb_0009.jpg
20200811_wooden_rgb_0001.xml  20200811_wooden_rgb_0009.xml
20200811_wooden_rgb_0002.jpg  20200811_wooden_rgb_0010.jpg
20200811_wooden_rgb_0002.xml  20200811_wooden_rgb_0010.xml
20200811_wooden_rgb_0003.jpg  20200811_wooden_rgb_0011.jpg
20200811_wooden_rgb_0003.xml  20200811_wooden_rgb_0011.xml
20200811_wooden_rgb_0004.jpg  20200811_wooden_rgb_0012.jpg
20200811_wooden_rgb_0004.xml  20200811_wooden_rgb_0012.xml
20200811_wooden_rgb_0005.jpg  20200811_wooden_rgb_0013.jpg
20200811_wooden_rgb_0005.xml  20200811_wooden_rgb_0013.xml
20200811_wooden_rgb_0006.jpg  20200811_wooden_rgb_0014.jpg
20200811_wooden_rgb_0006.xml  20200811_wooden_rgb_0014.xml
20200811_wooden_rgb_0007.jpg  20200811_wooden_rgb_0015.jpg
20200811_wooden_rgb_0007.xml  20200811_wooden_rgb_0015.xml


In [4]:
%cd /home/opencv/src/models/research
!protoc object_detection/protos/*.proto --python_out=.

!python object_detection/builders/model_builder_test.py

/home/opencv/src/models/research
object_detection/protos/input_reader.proto:5:1: warning: Import object_detection/protos/image_resizer.proto is unused.


In [5]:
#%%capture
repo_dir_path = '/home/opencv/src/object_detection_demo_flow'
%cd {repo_dir_path}
print(os.getcwd())

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

# Set the paths
test_record_fname = '/home/opencv/src/object_detection_demo_flow/data/annotations/test.record'
train_record_fname = '/home/opencv/src/object_detection_demo_flow/data/annotations/train.record'
label_map_pbtxt_fname = '/home/opencv/src/object_detection_demo_flow/data/annotations/label_map.pbtxt'

/home/opencv/src/object_detection_demo_flow
/home/opencv/src/object_detection_demo_flow
Successfully converted xml to csv.
Generate `data/annotations/label_map.pbtxt`
Successfully converted xml to csv.


W0825 14:52:35.578174 140522857449280 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0825 14:52:35.588204 140522857449280 module_wrapper.py:139] From generate_tfrecord.py:53: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /home/opencv/src/object_detection_demo_flow/data/annotations/train.record


W0825 14:52:37.280171 139961004173120 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0825 14:52:37.287811 139961004173120 module_wrapper.py:139] From generate_tfrecord.py:53: The name tf.gfile.GFile is deprecated. Please use 

In [6]:
import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/home/opencv/src/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/home/opencv/src/models/research/pretrained_model
total 135M
drwxr-xr-x  3 opencv opencv 4.0K  3月 30  2018 .
drwxrwxr-x 64 opencv opencv 4.0K  8月 25 14:53 ..
-rw-r--r--  1 opencv opencv   77  3月 30  2018 checkpoint
-rw-r--r--  1 opencv opencv  67M  3月 30  2018 frozen_inference_graph.pb
-rw-r--r--  1 opencv opencv  65M  3月 30  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 opencv opencv  15K  3月 30  2018 model.ckpt.index
-rw-r--r--  1 opencv opencv 3.4M  3月 30  2018 model.ckpt.meta
-rw-r--r--  1 opencv opencv 4.2K  3月 30  2018 pipeline.config
drwxr-xr-x  3 opencv opencv 4.0K  3月 30  2018 saved_model


In [7]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/home/opencv/src/models/research/pretrained_model/model.ckpt'

# Configuring a Training Pipeline

In [8]:
import os
pipeline_fname = os.path.join('/home/opencv/src/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [9]:
import re
iou_threshold = 0.50
num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    # Set number of classes num_classes.
    s = re.sub('iou_threshold: [0-9].[0-9]+',
               'iou_threshold: {}'.format(iou_threshold), s)
    
    f.write(s)

# Train the model

In [10]:
#After running this cell click on the link in the output cell to open tensorboard
#Tensoarboard will show you graphically different training parameters as the model is training
#when training finishes after the set number of steps, tensorboard can be used to see a nice summary of the training process
#Visuals will load in Tensorboard after the model has gone through a few hundred steps

model_dir = '/home/opencv/src/object_detection_demo_flow/training20200825/'
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip

In [16]:
LOG_DIR = model_dir
print(model_dir)
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')


/home/opencv/src/object_detection_demo_flow/training20200825/


In [17]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://f0026015e9c1.ngrok.io


In [18]:

print("prova")

prova


In [19]:
# necessary
#*********************** change folder name adding timestamp TODO *************

# remove previous training result
!rm -rf ~/.nv
# !rm -rf {model_dir}
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Optionally remove content in output model directory for a fresh start.
# !rm -rf {model_dir}
# os.makedirs(model_dir, exist_ok=True)
!python /home/opencv/src/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

W0825 12:45:12.777291 140167842056000 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 5000
I0825 12:45:12.777461 140167842056000 config_util.py:552] Maybe overwriting train_steps: 5000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0825 12:45:12.777545 140167842056000 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0825 12:45:12.777610 140167842056000 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0825 12:45:12.777678 140167842056000 config_util.py:552] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0825 12:45:12.777738 140167842056000 config_util.py:552] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0825 12:45:12.777795 140167842056000 config_util.py:5

Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W0825 12:45:28.096990 140167842056000 api.py:332] From /home/opencv/anaconda2/envs/ssd/lib/python3.6/site-packages/tensorflow_core/python/autograph/operators/control_flow.py:1004: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf.cast` instead.
W0825 12:45:31.165759 140167842056000 deprecation.py:323] From /home/opencv/src/models/research/object_detection/inputs.py:258: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
I0825 12:45:33.846473 140167842056000 estimator.py:1148] Calling model_fn.
Instructions for updating:
Please use `layer.__call__`

INFO:tensorflow:Running local_init_op.
I0825 12:45:53.906576 140167842056000 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0825 12:45:54.335668 140167842056000 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /home/opencv/src/object_detection_demo_flow/training20200825/model.ckpt.
I0825 12:46:03.146496 140167842056000 basic_session_run_hooks.py:606] Saving checkpoints for 0 into /home/opencv/src/object_detection_demo_flow/training20200825/model.ckpt.
2020-08-25 12:46:12.990530: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-08-25 12:46:16.822217: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
INFO:tensorflow:loss = 20.137066, step = 0
I0825 12:46:18.692188 140167842056000 basic_session_run_hooks.py:262] loss = 20.137066, step = 0
INFO:tensorflow:global

INFO:tensorflow:Done calling model_fn.
I0825 12:56:10.182485 140167842056000 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-25T12:56:10Z
I0825 12:56:10.194294 140167842056000 evaluation.py:255] Starting evaluation at 2020-08-25T12:56:10Z
INFO:tensorflow:Graph was finalized.
I0825 12:56:10.486029 140167842056000 monitored_session.py:240] Graph was finalized.
2020-08-25 12:56:10.486445: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-25 12:56:10.486848: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2060 SUPER major: 7 minor: 5 memoryClockRate(GHz): 1.68
pciBusID: 0000:05:00.0
2020-08-25 12:56:10.486931: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there mus

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1233: /home/opencv/src/object_detection_demo_flow/training20200825/model.ckpt-1233
I0825 12:56:14.729385 140167842056000 estimator.py:2109] Saving 'checkpoint_path' summary for global step 1233: /home/opencv/src/object_detection_demo_flow/training20200825/model.ckpt-1233
INFO:tensorflow:global_step/sec: 1.76019
I0825 12:56:46.912713 140167842056000 basic_session_run_hooks.py:692] global_step/sec: 1.76019
INFO:tensorflow:loss = 1.1832302, step = 1300 (56.812 sec)
I0825 12:56:46.913241 140167842056000 basic_session_run_hooks.py:260] loss = 1.1832302, step = 1300 (56.812 sec)
INFO:tensorflow:global_step/sec: 2.10598
I0825 12:57:34.396586 140167842056000 basic_session_run_hooks.py:692] global_step/sec: 2.10598
INFO:tensorflow:loss = 1.0736593, step = 1400 (47.484 sec)
I0825 12:57:34.397140 140167842056000 basic_session_run_hooks.py:260] loss = 1.0736593, step = 1400 (47.484 sec)
INFO:tensorflow:global_step/sec: 2.11397
I0825 

...

INFO:tensorflow:Calling model_fn.
I0825 13:16:07.887979 140167842056000 estimator.py:1148] Calling model_fn.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0825 13:16:09.412483 140167842056000 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0825 13:16:09.434355 140167842056000 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0825 13:16:09.456292 140167842056000 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0825 13:16:09.477833 140167842056000 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0825 13:16:09.499735 140167842056000 convolutional_box_predictor.py:156] 

INFO:tensorflow:global_step/sec: 1.81391
I0825 13:16:43.381556 140167842056000 basic_session_run_hooks.py:692] global_step/sec: 1.81391
INFO:tensorflow:loss = 0.8577164, step = 3800 (55.130 sec)
I0825 13:16:43.382174 140167842056000 basic_session_run_hooks.py:260] loss = 0.8577164, step = 3800 (55.130 sec)
INFO:tensorflow:global_step/sec: 2.11246
I0825 13:17:30.719616 140167842056000 basic_session_run_hooks.py:692] global_step/sec: 2.11246
INFO:tensorflow:loss = 0.87485516, step = 3900 (47.338 sec)
I0825 13:17:30.720156 140167842056000 basic_session_run_hooks.py:260] loss = 0.87485516, step = 3900 (47.338 sec)
INFO:tensorflow:global_step/sec: 2.11279
I0825 13:18:18.050370 140167842056000 basic_session_run_hooks.py:692] global_step/sec: 2.11279
INFO:tensorflow:loss = 0.9864846, step = 4000 (47.331 sec)
I0825 13:18:18.050912 140167842056000 basic_session_run_hooks.py:260] loss = 0.9864846, step = 4000 (47.331 sec)
INFO:tensorflow:global_step/sec: 2.11656
I0825 13:19:05.296905 14016784205

INFO:tensorflow:Running local_init_op.
I0825 13:26:12.364624 140167842056000 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0825 13:26:12.481806 140167842056000 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Performing evaluation on 3 images.
I0825 13:26:14.624359 140162407319296 coco_evaluation.py:237] Performing evaluation on 3 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I0825 13:26:14.624958 140162407319296 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
I0825 13:26:14.625843 140162407319296 coco_tools.py:138] DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50      | area=   a

INFO:tensorflow:Running local_init_op.
I0825 13:26:24.184269 140167842056000 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0825 13:26:24.299682 140167842056000 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Performing evaluation on 3 images.
I0825 13:26:26.406959 140162398926592 coco_evaluation.py:237] Performing evaluation on 3 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I0825 13:26:26.407395 140162398926592 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
I0825 13:26:26.408317 140162398926592 coco_tools.py:138] DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.283
 Average Precision  (AP) @[ IoU=0.50      | area=   a

INFO:tensorflow:Assets added to graph.
I0825 13:26:29.374114 140167842056000 builder_impl.py:665] Assets added to graph.
INFO:tensorflow:No assets to write.
I0825 13:26:29.374247 140167842056000 builder_impl.py:460] No assets to write.
INFO:tensorflow:SavedModel written to: /home/opencv/src/object_detection_demo_flow/training20200825/export/Servo/temp-b'1598329586'/saved_model.pb
I0825 13:26:29.933962 140167842056000 builder_impl.py:425] SavedModel written to: /home/opencv/src/object_detection_demo_flow/training20200825/export/Servo/temp-b'1598329586'/saved_model.pb
INFO:tensorflow:Loss for final step: 0.884982.
I0825 13:26:30.307108 140167842056000 estimator.py:371] Loss for final step: 0.884982.


In [11]:
!pwd

/home/opencv/src/object_detection_demo_flow


In [11]:
import re
import numpy as np

print("test")
output_directory = '/home/opencv/src/object_detection_demo_flow/frozen_model'
# output_directory = '/content/gdrive/My\ Drive/data/'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python ~/src/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

test
/home/opencv/src/object_detection_demo_flow/training20200825/model.ckpt-5000
Instructions for updating:
Please use `layer.__call__` method instead.
W0825 14:53:35.907724 140625065346880 deprecation.py:323] From /home/opencv/anaconda2/envs/ssd/lib/python3.6/site-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0825 14:53:37.502573 140625065346880 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0825 14:53:37.530047 140625065346880 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0825 14:53:37.556880 140625065346880 convolutional_box_predictor

139 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz

2020-08-25 14:53:39.690921: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-25 14:53:39.738593: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-25 14:53:39.739055: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2060 SUPER major: 7 minor: 5 memoryClockRate(GHz): 1.68
pciBusID: 0000:05:00.0
2020-08-25 14:53:39.739107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-25 14:53:39.739395: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce GT 710 major: 3 minor: 5 memoryClockRate(GHz): 0.954
pciBusID: 0000:01:0

2020-08-25 14:53:41.788620: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-25 14:53:41.788964: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2060 SUPER major: 7 minor: 5 memoryClockRate(GHz): 1.68
pciBusID: 0000:05:00.0
2020-08-25 14:53:41.789009: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-08-25 14:53:41.789025: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-08-25 14:53:41.789040: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2020-08-25 14:53:41.789053: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library 

INFO:tensorflow:SavedModel written to: /home/opencv/src/object_detection_demo_flow/frozen_model/saved_model/saved_model.pb
I0825 14:53:43.471837 140625065346880 builder_impl.py:425] SavedModel written to: /home/opencv/src/object_detection_demo_flow/frozen_model/saved_model/saved_model.pb
INFO:tensorflow:Writing pipeline config file to /home/opencv/src/object_detection_demo_flow/frozen_model/pipeline.config
I0825 14:53:43.483418 140625065346880 config_util.py:254] Writing pipeline config file to /home/opencv/src/object_detection_demo_flow/frozen_model/pipeline.config


In [12]:
import os
print("test1")
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)
# !ls -alh {pb_fname}

test1


In [23]:
print(pb_fname)

/home/opencv/src/object_detection_demo_flow/frozen_model/frozen_inference_graph.pb


In [16]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "data/images/final_test")

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

['/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0617.jpg', '/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0616.jpg', '/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0622.jpg', '/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0614.jpg', '/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0623.jpg', '/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0613.jpg', '/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0620.jpg', '/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0615.jpg', '/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0619.jpg', '/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0621.jpg', '/home/op

In [17]:
%cd ~/src/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
# This is needed to display the images.
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

print("test")
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

/home/opencv/src/models/research/object_detection
test


In [18]:
image_path = "/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0613.jpg"
image = Image.open(image_path)
print(image_path)
image = Image.open(image_path)
# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)
# Actual detection.
output_dict = run_inference_for_single_image(image_np, detection_graph)
# Visualization of the results of a detection.
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index,
    instance_masks=output_dict.get('detection_masks'),
    use_normalized_coordinates=True,
    line_thickness=8)
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)
plt.show()

/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0613.jpg


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node FeatureExtractor/MobilenetV2/Conv/Conv2D (defined at /home/opencv/anaconda2/envs/ssd/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
	 [[Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Reshape_2/_91]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node FeatureExtractor/MobilenetV2/Conv/Conv2D (defined at /home/opencv/anaconda2/envs/ssd/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'FeatureExtractor/MobilenetV2/Conv/Conv2D':
  File "/home/opencv/anaconda2/envs/ssd/lib/python3.6

In [ ]:
for image_path in TEST_IMAGE_PATHS:
    print(image_path)
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    plt.show()

/home/opencv/src/models/research/object_detection
test
/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0617.jpg
/home/opencv/src/object_detection_demo_flow/data/images/final_test/20200812_wooden_rgb_0617.jpg


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node FeatureExtractor/MobilenetV2/Conv/Conv2D (defined at /home/opencv/anaconda2/envs/ssd/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
	 [[Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Reshape_2/_91]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node FeatureExtractor/MobilenetV2/Conv/Conv2D (defined at /home/opencv/anaconda2/envs/ssd/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'FeatureExtractor/MobilenetV2/Conv/Conv2D':
  File "/home/opencv/anaconda2/envs/ssd/lib/python3.6